In [1]:
import tensorflow as tf
import gym
import numpy as np
from collections import deque

offene Probleme / mögliche Verbesserungen:
    - in einem Endzustand nicht mehr q(a',s') berechnen
    - Minibatch mit TF erzeugen
    - store old network weights and use them in loss function

In [ ]:
# build graph
obs_dim = 4
n_acts = 2
gamma = 0.99

obs_ph = tf.placeholder(shape=(None, obs_dim), dtype=tf.float32)
act_ph = tf.placeholder(shape=(None, 1), dtype=tf.int32)
rew_ph = tf.placeholder(shape=(None, 1), dtype=tf.float32)
new_obs_ph = tf.placeholder(shape=(None, obs_dim), dtype=tf.float32)
terminal_ph = tf.placeholder(shape=(None, 1), dtype=tf.float32)

# make core of state-action-value function network
mlp_action_val = tf.keras.models.Sequential()
mlp_action_val.add(tf.keras.layers.Dense(50, activation='tanh'))
mlp_action_val.add(tf.keras.layers.Dense(50, activation='tanh'))
mlp_action_val.add(tf.keras.layers.Dense(n_acts))

# define state action values
old_state_action_values = mlp_action_val(obs_ph)
new_state_action_values = mlp_action_val(new_obs_ph)

# select action
greedy_action = tf.math.argmax(old_state_action_values)

# define loss function
y = rew_ph + gamma * tf.reduce_max(new_state_action_values, axis=0)*(1-terminal_ph)
y = tf.stop_gradient(y) 
action_masks = tf.one_hot(act_ph, n_acts)
old_selected_action_values = tf.reduce_sum(action_masks * old_state_action_values, axis=1)
action_value_loss = tf.reduce_mean((y - old_selected_action_values)**2)

# define optimizer
optimizer_action_value = tf.train.AdamOptimizer(0.001)
train_action_value = optimizer_action_value.minimize(action_value_loss)


In [ ]:
def dqn(sess, env, experience_replay_buffer, n_episodes):
    for i in range(n_episodes):
        done = False
        state = env.reset()
        cnt = 0
        while not done:
            action = select_action(sess, state)
            new_state, reward, done, _ = env.step(action)
            experience_replay_buffer.append((state, action, reward, new_state, float(done)))
            state = new_state
            for _ in range(1):
                update_q_function(experience_replay_buffer)
            cnt += 1
            
            
        if i % 10 == 0:
            print(i, cnt)
            print(sess.run([old_state_action_values], feed_dict={obs_ph: np.zeros(4).reshape(-1,4)}))
            

            

        
def update_q_function(experience_replay_buffer):
    experience_batch = sample_experience_batch(experience_replay_buffer)
    states, actions, rewards, new_states, terminal_flags = extract_data(experience_batch)
    loss = gradient_step(states, actions, rewards, new_states, terminal_flags)
#     if np.random.rand() < 0.01:
#         print(loss)

def gradient_step(states, actions, rewards, new_states, terminal_flags):
    _, loss, action_values = sess.run([train_action_value, action_value_loss, old_state_action_values], feed_dict={
                                obs_ph: np.array(states).reshape(-1, obs_dim),
                                act_ph: np.array(actions).reshape(-1,1),
                                rew_ph: np.array(rewards).reshape(-1,1),
                                new_obs_ph: np.array(new_states).reshape(-1,obs_dim),
                                terminal_ph: np.array(terminal_flags).reshape(-1,1)
                             })
#     print(action_values[:1])
    return loss
    
def extract_data(experience_batch):
    states = [x[0] for x in experience_batch]
    actions = [x[1] for x in experience_batch]
    rewards = [x[2] for x in experience_batch]
    new_states = [x[3] for x in experience_batch]
    terminal_flags = [x[4] for x in experience_batch]
    return states, actions, rewards, new_states, terminal_flags
    
def sample_experience_batch(experience_replay_buffer, batch_size=100):
    indices = np.random.choice(np.arange(len(experience_replay_buffer)), batch_size)
    tmp = np.array(experience_replay_buffer)
    sample_batch = tmp[indices, :]
#     sample_batch = np.random.choice(experience_replay_buffer,batch_size, replace = False)
    return sample_batch

def select_action(sess, state):
    if np.random.rand() < 0.5:
        action = np.random.randint(2)
    else:
        action = sess.run(greedy_action, {obs_ph: state.reshape(1,-1)})[0]
    return action

In [ ]:
# main
exp_replay_size = 5000
experience_replay_buffer = deque([], exp_replay_size)
env = gym.make('CartPole-v0')

sess = tf.Session()
sess.run(tf.global_variables_initializer())

dqn(sess, env, experience_replay_buffer, 100000)

0 19
[array([[ 0.04023315, -0.03938903]], dtype=float32)]
10 17
[array([[ 0.02224731, -0.04316805]], dtype=float32)]
20 17
[array([[ 0.01286637, -0.02037151]], dtype=float32)]
30 20
[array([[ 0.01352159, -0.0027991 ]], dtype=float32)]
40 9
[array([[0.01060689, 0.00730381]], dtype=float32)]
50 12
[array([[0.0155193 , 0.01095267]], dtype=float32)]
60 12
[array([[0.01135833, 0.01115458]], dtype=float32)]
70 17
[array([[0.00872821, 0.01033343]], dtype=float32)]
80 13
[array([[0.00781166, 0.01222477]], dtype=float32)]
90 11
[array([[0.01020014, 0.01211565]], dtype=float32)]
100 24
[array([[0.00975416, 0.00971564]], dtype=float32)]
110 27
[array([[0.01195613, 0.00954548]], dtype=float32)]
120 15
[array([[0.01003951, 0.01039715]], dtype=float32)]
130 16
[array([[0.01092268, 0.0144839 ]], dtype=float32)]
140 9
[array([[0.01830473, 0.01524862]], dtype=float32)]
150 9
[array([[0.00707266, 0.00878637]], dtype=float32)]
160 12
[array([[0.00983759, 0.00999003]], dtype=float32)]
170 15
[array([[0.01

1410 23
[array([[0.00982023, 0.01067843]], dtype=float32)]
1420 11
[array([[0.01014396, 0.01067397]], dtype=float32)]
1430 14
[array([[0.01097031, 0.00825865]], dtype=float32)]
1440 13
[array([[0.01030145, 0.01009226]], dtype=float32)]
1450 16
[array([[0.01052278, 0.01324765]], dtype=float32)]
1460 12
[array([[0.01066408, 0.00243779]], dtype=float32)]
1470 12
[array([[0.01247116, 0.01100477]], dtype=float32)]
1480 11
[array([[0.00983712, 0.01007433]], dtype=float32)]
1490 10
[array([[0.01044063, 0.01009675]], dtype=float32)]
1500 10
[array([[0.01060212, 0.01062958]], dtype=float32)]
1510 8
[array([[0.01007883, 0.01048711]], dtype=float32)]
1520 12
[array([[0.01035045, 0.01030118]], dtype=float32)]
1530 12
[array([[0.01026078, 0.01230935]], dtype=float32)]
1540 20
[array([[0.01064557, 0.01466586]], dtype=float32)]
1550 12
[array([[0.00764482, 0.0105586 ]], dtype=float32)]
1560 15
[array([[0.01068998, 0.00877973]], dtype=float32)]
1570 16
[array([[0.01028119, 0.01055161]], dtype=float32)

2800 15
[array([[0.01037549, 0.01001064]], dtype=float32)]
2810 10
[array([[0.01011664, 0.01040848]], dtype=float32)]
2820 14
[array([[0.01006561, 0.01044523]], dtype=float32)]
2830 15
[array([[0.01096247, 0.01013316]], dtype=float32)]
2840 14
[array([[0.00860699, 0.01006175]], dtype=float32)]
2850 9
[array([[0.0101968 , 0.00998855]], dtype=float32)]
2860 9
[array([[0.0100954 , 0.01010604]], dtype=float32)]
2870 10
[array([[0.01013396, 0.0100766 ]], dtype=float32)]
2880 20
[array([[0.0100694 , 0.01012463]], dtype=float32)]
2890 13
[array([[0.01011907, 0.01016765]], dtype=float32)]
2900 13
[array([[0.01008802, 0.01141151]], dtype=float32)]
2910 18
[array([[0.01010373, 0.01104147]], dtype=float32)]
2920 9
[array([[0.00981464, 0.00983757]], dtype=float32)]
2930 15
[array([[0.01008564, 0.01039931]], dtype=float32)]
2940 11
[array([[0.01016628, 0.01018556]], dtype=float32)]
2950 12
[array([[0.01003537, 0.00992765]], dtype=float32)]
2960 11
[array([[0.00997538, 0.01033082]], dtype=float32)]
